In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
parks = pd.read_csv('archive-2/parks.csv')
species = pd.read_csv('archive-2/species.csv')

/var/folders/ly/1tqv92455b5db15clb7wtwwh0000gn/T/ipykernel_6505/549659019.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  species = pd.read_csv('archive-2/species.csv')


In [4]:
#species['scientific_name']

In [5]:
parks.shape

(56, 6)

In [13]:
parks['Park Code'].to_csv('park_codes.csv')

In [11]:
parks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Park Code  56 non-null     object 
 1   Park Name  56 non-null     object 
 2   State      56 non-null     object 
 3   Acres      56 non-null     int64  
 4   Latitude   56 non-null     float64
 5   Longitude  56 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 2.8+ KB


In [6]:
species.shape

(119248, 14)

In [10]:
species['category'].value_counts()

category
Vascular Plant         65221
Bird                   14601
Insect                 14349
Fungi                   6203
Nonvascular Plant       4278
Fish                    3956
Mammal                  3867
Invertebrate            1566
Reptile                 1343
Algae                    976
Slug/Snail               787
Spider/Scorpion          776
Amphibian                743
Crab/Lobster/Shrimp      582
Name: count, dtype: int64

In [15]:
parks = parks.rename(columns={'Park Code': 'park_code', 'Park Name': 'park_name', 'State': 'state', 'Acres': 'acres', 'Latitude':'latitude', 'Longitude': 'longitude'})

In [19]:
parks['area_km2'] = round(parks['acres']/247.105,0)

In [20]:
parks

,park_code,park_name,state,acres,latitude,longitude,area_km2
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,192.0
1,ARCH,Arches National Park,UT,76519,38.68,-109.57,310.0
2,BADL,Badlands National Park,SD,242756,43.75,-102.50,982.0
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25,3242.0
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08,700.0
5,BLCA,Black Canyon of the Gunnison National Park,CO,32950,38.57,-107.72,133.0
6,BRCA,Bryce Canyon National Park,UT,35835,37.57,-112.18,145.0
7,CANY,Canyonlands National Park,UT,337598,38.20,-109.93,1366.0
8,CARE,Capitol Reef National Park,UT,241904,38.20,-111.17,979.0
9,CAVE,Carlsbad Caverns National Park,NM,46766,32.17,-104.44,189.0


In [24]:
#we drop this column because it's empty
species = species.drop(columns='Unnamed: 13')

In [25]:
species.columns

Index(['species_id', 'park_name', 'category', 'order', 'family',
       'scientific_name', 'common_names', 'record_status', 'occurrence',
       'nativeness', 'abundance', 'seasonality', 'conservation_status'],
      dtype='object')